In [ ]:
import pandas as pd
df = pd.read_csv('C:/project/physionet2012/physionet2012_train.csv', encoding='cp949')

### 타겟 컬럼 추가 및 이상 타겟 제거, NA 컬럼 제거

In [ ]:
# 타겟 컬럼 추가
df['target'] = 4

df.loc[df['survival']>df['LOS'], 'target'] = 0
df.loc[df['survival']<=df['LOS'], 'target'] = 1
df.loc[df['survival']==-1, 'target'] = 0
df.loc[df['survival']==0, 'target'] = 3
df.loc[df['survival']==1, 'target'] = 3

In [139]:
# target값이 3인사람 제거
print(len(df[df['target']==3]['pid'].unique()),'명의 pid가 있습니다.')
print(df[df['target']==3].shape[0],'개의 데이터가 있습니다.')
df = df.drop(df[df['target']==3].index)

24 명의 pid가 있습니다.
1963 개의 데이터가 있습니다.


In [140]:
# gender가 -1인 사람 제거
print(len(df[df['Gender']==-1]['pid'].unique()),'명의 pid가 있습니다.')
print(len(df[df['Gender']==-1]),'개의 데이터가 있습니다.')
df = df.drop(df[df['Gender']==-1].index)

2 명의 pid가 있습니다.
68 개의 데이터가 있습니다.


In [141]:
print(len(df['pid'].unique()), '명의 데이터로 분석진행')

3214 명의 데이터로 분석진행


In [142]:
# 인덱스값 초기화
df = df.reset_index(drop=True)

In [143]:
# 사용하지 않는 6개 컬럼 제거
df = df.drop(['Unnamed: 0','male', 'female', 'surv_los','survival','LOS'], axis = 1)

In [144]:
# missing value 에서 찾은 90%이상 na인 데이터 6개는 제거

df = df.drop(['pH', 'PaCO2', 'PaO2', 'HCT', 'K', 'Platelets', 'Creatinine', 'BUN',
 'Mg', 'HCO3', 'Na', 'Glucose', 'WBC', 'Lactate', 'AST', 'Bilirubin', 'ALT',
 'ALP', 'Albumin', 'TroponinT', 'TroponinI', 'Cholesterol', 'TropT', 'Tropl'], axis=1)

### 결측치 보간

In [148]:
df[['Urine', 'GCS', 'FiO2', 'MechVent']].describe()

,Urine,GCS,FiO2,MechVent
count,109110.000000,49575.000000,25860.000000,24843.0
mean,119.661232,11.443651,0.544407,1.0
std,181.727151,3.945192,0.189474,0.0
min,0.000000,3.000000,0.210000,1.0
25%,35.000000,8.000000,0.400000,1.0
50%,70.000000,13.000000,0.500000,1.0
75%,140.000000,15.000000,0.600000,1.0
max,11000.000000,15.000000,1.000000,1.0


In [149]:
#결측치는 모두 중앙값으로 대체
df['Urine'].fillna(70, inplace=True)
df['GCS'].fillna(13, inplace=True)
df['FiO2'].fillna(0.5, inplace=True)
df['MechVent'].fillna(1, inplace=True)

In [150]:
len(df['pid'].unique())

3214

### 데이터 개수 통일시키기

In [151]:
# 몇개로 통일시킬지 확인하기 위해 percentile로 분포 확인
import numpy as np
print(np.percentile(df['pid'].value_counts(),10))
print(np.percentile(df['pid'].value_counts(),50))
print(np.percentile(df['pid'].value_counts(),90))

#중앙 값보다 조금 더 낮은 70으로 통일 시키자!

52.0
72.0
104.0


In [152]:
import copy
last_index = copy.deepcopy(df['pid'][len(df['pid'])-1])
# last_index = 마지막 행의 pid 값

In [153]:
# 70개보다 크면 drop, 작으면 더미행 합치기

i = 0 
while True: #하단 break 조건 되기 전엔 무한반복
    print(i)    # i를 찍어 실제 엑살 파일과 동일한 위치에서 구분되는지 확인
    cnt = 0     # 빈 데이터프레임의 갯수를 세기 위한 변수 cnt
    while len(df[df['pid']==df['pid'][i]]) != 70:   # i번째 pid의 갯수가 70개가 아니라면 이하 반복
        if len(df[df['pid']==df['pid'][i]]) > 70:       # 만약 i번째 pid의 갯수가 70보다 크다면
            df = df.drop(i)                                 # i는 삭제한다
            i+=1                                            # 다음 작업을 위해 i = i + 1
        else:                                           # 아니라면 (70보다 작다면)
            pre_dummy1 = pd.DataFrame(np.zeros((1,15)))     # 0으로 채워진 더미를 만들고
            pre_dummy2 = pd.concat([pd.DataFrame([df['pid'][i]]),pd.DataFrame(['0:00:00'])], axis = 1)  # [피드+시간] 더미 만들고
            dummy = pd.concat([pre_dummy2, pre_dummy1], axis=1) # 합쳐서(1,17)로 형태를 갖춰줌
            dummy.columns = df.columns                          # concat을 위한 column명 통일
            df = pd.concat([df,dummy], axis = 0)                # 원래 데이터프레임에 더미 데이터프레임 통합 = 패딩
            cnt+=1                                              # 빈 데이터프레임의 갯수 : cnt
        
    i+=(70-cnt) # i = i + 70 - cnt == 삭제한 pid수(i) + 남은pid수(70) - 추가한 pid수(cnt)

    try:    
        if len(df[df['pid']==last_index])==70:  # 마지막 pid의 길이 == 70 이면
            break                               # 종료!!!!!
    except:
        pass


0
80
184
253
316
414
484
617
722
776
979
1091
1169
1219
1274
1330
1392
1453
1554
1607
1660
1746
1785
1905
1987
2066
2153
2232
2319
2409
2493
2556
2614
2673
2752
2805
2861
2914
2972
3028
3090
3170
3231
3286
3343
3394
3460
3464
3547
3655
3715
3817
3832
3874
3927
4060
4138
4227
4294
4297
4408
4493
4552
4633
4730
4809
4888
4992
5072
5150
5225
5335
5403
5461
5557
5656
5726
5865
5927
6019
6132
6185
6250
6293
6363
6485
6575
6644
6687
6745
6839
6893
6961
7045
7168
7278
7355
7464
7548
7604
7677
7751
7918
7992
8069
8156
8225
8288
8357
8415
8461
8524
8616
8692
8764
8851
8966
9050
9148
9238
9294
9369
9430
9511
9671
9729
9801
9853
9914
9994
10069
10144
10195
10248
10294
10408
10494
10547
10618
10671
10741
10800
10868
10946
11003
11064
11114
11207
11286
11369
11463
11541
11623
11680
11755
11830
11915
11998
12053
12106
12191
12278
12343
12392
12488
12545
12614
12675
12725
12784
12851
12950
13007
13063
13140
13211
13302
13392
13473
13542
13620
13733
13815
13964
14030
14075
14161
14223
14261
14333
1438

In [165]:
df = df.sort_values(by=['pid','rec_time'])

In [172]:
# df = df.reset_index(drop=True)

In [16]:
df.to_csv('C:/project/physionet2012/padded_train_data_remove_tar_gen.csv', encoding='cp949', index=False)